In [1]:
%pwd

'/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer/research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer'

In [ ]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path 
    data_path: Path 
    tokenizer_name: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from src.textSummarizer.logging import logger


In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_path=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        self.config = read_yaml(config_path)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config=self.config.data_trasnformation
        create_directories([config.root_dir])
        logger.info(f"Data Transformation config: {config}")
        logger.info(f"Data Transformation params: {config.data_path}")
        logger.info(f"{config.tokenizer_name}")
        logger.info(f"Data Transformation root dir: {config.root_dir}")
        return DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )

In [8]:
import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.


In [9]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer= AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    def convert_examples_to_features(self,example_batch):
        logger.info(f"Converting examples to features")
        input_encodings = self.tokenizer(example_batch['dialogue'] , max_length = 1024, truncation = True )

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True )

        return {
            'input_ids' : input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }
    
    def convert(self):
        logger.info(f"Loading dataset from {self.config.data_path}")
        dataset_samsum= load_from_disk(self.config.data_path)
        logger.info(f"Dataset loaded successfully with {len(dataset_samsum)} records")
        dataset_samsum_pt= dataset_samsum.map(
            self.convert_examples_to_features,
            batched=True
        )
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))

In [10]:
config=ConfigurationManager()
config_data_transformation_config=config.get_data_transformation_config()
data_transformation=DataTransformation(config_data_transformation_config)
data_transformation.convert()

2025-06-18 20:58:44,319 - INFO - yaml file: config/config.yaml loaded successfully
2025-06-18 20:58:44,320 - INFO - yaml file: params.yaml loaded successfully
2025-06-18 20:58:44,321 - INFO - created directory at: artifacts
2025-06-18 20:58:44,321 - INFO - created directory at: artifacts/data_transformation
2025-06-18 20:58:44,321 - INFO - Data Transformation config: {'root_dir': 'artifacts/data_transformation', 'data_path': 'artifacts/data_ingestion/samsum_dataset', 'tokenizer_name': 'google/pegasus-cnn_dailymail'}
2025-06-18 20:58:44,322 - INFO - Data Transformation params: artifacts/data_ingestion/samsum_dataset
2025-06-18 20:58:44,322 - INFO - google/pegasus-cnn_dailymail
2025-06-18 20:58:44,322 - INFO - Data Transformation root dir: artifacts/data_transformation
2025-06-18 20:58:45,268 - INFO - Loading dataset from artifacts/data_ingestion/samsum_dataset
2025-06-18 20:58:45,280 - INFO - Dataset loaded successfully with 3 records


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

2025-06-18 20:58:45,292 - INFO - Converting examples to features
2025-06-18 20:58:45,380 - INFO - Converting examples to features


/Users/rajaanthonysamy/my_ml_projects/summarizer/TextSummarizer/venv/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3959: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Map:  14%|█▎        | 2000/14732 [00:00<00:01, 10510.67 examples/s]

2025-06-18 20:58:45,482 - INFO - Converting examples to features
2025-06-18 20:58:45,569 - INFO - Converting examples to features


...

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

2025-06-18 20:58:46,594 - INFO - Converting examples to features


Map:   0%|          | 0/818 [00:00<?, ? examples/s]

2025-06-18 20:58:46,676 - INFO - Converting examples to features


Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 378023.43 examples/s]


In [11]:
dataset_samsum = load_from_disk('artifacts/data_ingestion/samsum_dataset')

In [12]:
dataset_samsum

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})